![](assets/mpcs56420.jpeg)

# BLASTN Search

A plasmid is a small, extrachromosomal DNA molecule within a cell that is physically separated from chromosomal DNA and can replicate independently. They are most commonly found as small circular, double-stranded DNA molecules in bacteria; however, plasmids are sometimes present in archaea and eukaryotic organisms.

In this notebook we run a BLASTN search using an antibiotic resistance gene (MH168512) as the query and a set of _E. coli_ plasmids as the database. We'll then use Python and the Pandas module to explore the resulting data.

## Download Sequence Database
Retrieving the plasmid sequences from NCBI in FASTA format.

In [3]:
!wget https://ftp.ncbi.nlm.nih.gov/blast/demo/Plasmids_562.fsa -P downloads/

--2020-10-21 14:34:59--  https://ftp.ncbi.nlm.nih.gov/blast/demo/Plasmids_562.fsa
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 2607:f220:41e:250::13, 2607:f220:41e:250::10, 2607:f220:41e:250::7, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|2607:f220:41e:250::13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131433412 (125M)
Saving to: ‘downloads/Plasmids_562.fsa’

Plasmids_562.fsa    100%[===================>] 125.34M   947KB/s    in 2m 26s  

2020-10-21 14:37:27 (879 KB/s) - ‘downloads/Plasmids_562.fsa’ saved [131433412/131433412]



## Build a BLAST database

Build a BLAST database, Plasmids_562, from the plasmid FASTA file and set the taxid (taxonomy) for every entry to _E. coli_ (id 562).

In [8]:
!makeblastdb -in downloads/Plasmids_562.fsa -dbtype nucl -parse_seqids -taxid 562 -out db/Plasmids_562



Building a new DB, current time: 10/21/2020 14:55:35
New DB name:   /Users/tabinkowski/Desktop/BLAST/assignment4/db/Plasmids_562
New DB title:  downloads/Plasmids_562.fsa
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1609 sequences in 1.66254 seconds.


## Download the Query Seqeunce
Download the query file for the antibiotic resistance gene.

In [9]:
!wget https://ftp.ncbi.nlm.nih.gov/blast/demo/MH168512.fsa -P downloads

--2020-10-21 14:56:09--  https://ftp.ncbi.nlm.nih.gov/blast/demo/MH168512.fsa
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 2607:f220:41e:250::7, 2607:f220:41e:250::12, 2607:f220:41e:250::11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|2607:f220:41e:250::7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 933
Saving to: ‘downloads/MH168512.fsa’

MH168512.fsa        100%[===================>]     933  --.-KB/s    in 0s      

2020-10-21 14:56:10 (28.7 MB/s) - ‘downloads/MH168512.fsa’ saved [933/933]



## Run BLASTN
Run a BLASTN search and format the results as a table.  The query is our antibiotic resistance gene (MH168512) and the database is the set of plasmid sequences. 

Note that we are setting a specific output format that will allow us to easily important the results into a pandas dataframe.

In [12]:
!blastn -db db/Plasmids_562 -query downloads/MH168512.fsa -outfmt "6 qseqid sseqid stitle pident qcovs length mismatch gapopen qstart qend sstart send qframe sframe frames evalue bitscore qseq sseq" -out MH168512.tab -max_target_seqs 5000

In [13]:
# Check the output of the tabular results with the UNIX "head" command just to make 
# sure something is there:
!head MH168512.tab

MH168512.2	ref|NZ_CP031235.1|	Escherichia coli strain Es_ST410_NW1_NDM_09_2017 plasmid pEsST410_NW_NDM, complete sequence	100.000	100	813	0	0	1	813	34633	35445	1	1	1/1	0.0	1502	ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCGCATTAGCCGCTGCATTGATGCTGAGCGGGTGCATGCCCGGTGAAATCCGCCCGACGATTGGCCAGCAAATGGAAACTGGCGACCAACGGTTTGGCGATCTGGTTTTCCGCCAGCTCGCACCGAATGTCTGGCAGCACACTTCCTATCTCGACATGCCGGGTTTCGGGGCAGTCGCTTCCAACGGTTTGATCGTCAGGGATGGCGGCCGCGTGCTGGTGGTCGATACCGCCTGGACCGATGACCAGACCGCCCAGATCCTCAACTGGATCAAGCAGGAGATCAACCTGCCGGTCGCGCTGGCGGTGGTGACTCACGCGCATCAGGACAAGATGGGCGGTATGGACGCGCTGCATGCGGCGGGGATTGCGACTTATGCCAATGCGTTGTCGAACCAGCTTGCCCCGCAAGAGGGGATGGTTGCGGCGCAACACAGCCTGACTTTCGCCGCCAATGGCTGGGTCGAACCAGCAACCGCGCCCAACTTTGGCCCGCTCAAGGTATTTTACCCCGGCCCCGGCCACACCAGTGACAATATCACCGTTGGGATCGACGGCACCGACATCGCTTTTGGTGGCTGCCTGATCAAGGACAGCAAGGCCAAGTCGCTCGGCAATCTCGGTGATGCCGACACTGAGCACTACGCCGCGTCAGCGCGCGCGTTTGGTGCGGCGTTCCCCAAGGCCAGCATGATCGTGATGAGCCATTCCGCCCCCGATAGCCGCGCCGCAATCACTCATACGGCCCGCATGGCCGACAAGCTGCGCTGA	ATGGAATTG

## Process the Results
Load the tabular results into a dataframe:

In [14]:
import pandas as pd

#load blast results into dataframe and rename columns with meaningful values
blast_results = pd.read_csv('MH168512.tab',
                           '\t',
                           header=None).rename(columns={0:'qseqid',
                                                        1:'sseqid',
                                                        2:'stitle',
                                                        3:'pident',
                                                        4:'qcovs',
                                                        5:'length',
                                                        6:'mismatch',
                                                        7:'gapopen',
                                                        8:'qstart',
                                                        9:'qend',
                                                        10:'sstart',
                                                        11:'send',
                                                        12:'qframe',
                                                        13:'sframe',
                                                        14:'frames',
                                                        15:'evalue',
                                                        16:'bitscore',
                                                        17:'qseq',
                                                        18:'sseq'})

# Reformat sseqid values so they are usuable for downstream operations
blast_results.sseqid = [x.split('|')[1] for x in blast_results.sseqid]
blast_results

,qseqid,sseqid,stitle,pident,qcovs,length,mismatch,gapopen,qstart,qend,sstart,send,qframe,sframe,frames,evalue,bitscore,qseq,sseq
0,MH168512.2,NZ_CP031235.1,Escherichia coli strain Es_ST410_NW1_NDM_09_20...,100.000,100,813,0,0,1,813,34633,35445,1,1,1/1,0.0,1502,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...
1,MH168512.2,NZ_CP034403.1,Escherichia coli strain CRE10 plasmid pCRE10.4...,100.000,100,813,0,0,1,813,21794,22606,1,1,1/1,0.0,1502,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...
2,MH168512.2,NZ_CP034398.1,"Escherichia coli strain CRE1 plasmid pCRE1.4, ...",100.000,100,813,0,0,1,813,21403,20591,1,-1,1/-1,0.0,1502,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...
3,MH168512.2,NZ_CP031297.1,Escherichia coli strain EC17GD31 plasmid pGD31...,100.000,100,813,0,0,1,813,127643,126831,1,-1,1/-1,0.0,1502,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...
4,MH168512.2,NZ_CM010662.1,Klebsiella pneumoniae strain Kp_SE1_NDM_10_201...,100.000,100,813,0,0,1,813,16063,15251,1,-1,1/-1,0.0,1502,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,MH168512.2,NZ_CM007910.1,Escherichia coli strain CH613_eco plasmid unna...,99.754,100,813,2,0,1,813,21052,21864,1,1,1/1,0.0,1491,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...
77,MH168512.2,NZ_CM007910.1,Escherichia coli strain CH613_eco plasmid unna...,99.508,100,813,2,1,1,813,123567,124377,1,1,1/1,0.0,1478,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...
78,MH168512.2,NZ_CP017981.1,Escherichia coli strain CH611_eco plasmid pGZ3...,99.754,100,813,2,0,1,813,53317,54129,1,1,1/1,0.0,1491,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...
79,MH168512.2,NZ_LS992188.1,Escherichia coli isolate Escherichia coli str....,99.631,100,813,3,0,1,813,70205,69393,1,-1,1/-1,0.0,1485,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...


We can use the dataframe to extract other information.  In the next two cells, we'll check how many rows are in the table, and how many unique database matches we found.

In [7]:
print('There are {} alignments, with {} unique subject sequences'.format(blast_results.index.size,
                                                                         blast_results.sseqid.unique().size))

# generate descriptive statistics for numerical columns
blast_results.describe()

There are 81 alignments, with 77 unique subject sequences


,pident,qcovs,length,mismatch,gapopen,qstart,qend,sstart,send,qframe,sframe,evalue,bitscore
count,81.000000,81.0,81.000000,81.000000,81.000000,81.000000,81.0,81.000000,81.000000,81.0,81.000000,81.0,81.000000
mean,99.852704,100.0,812.888889,1.160494,0.024691,1.111111,813.0,44596.740741,44566.740741,1.0,-0.037037,0.0,1495.123457
std,0.128150,0.0,1.000000,1.005694,0.156150,1.000000,0.0,40848.854130,40827.196349,0.0,1.005540,0.0,5.880865
min,99.508000,100.0,804.000000,0.000000,0.000000,1.000000,813.0,964.000000,152.000000,1.0,-1.000000,0.0,1478.000000
25%,99.754000,100.0,813.000000,0.000000,0.000000,1.000000,813.0,15694.000000,16506.000000,1.0,-1.000000,0.0,1491.000000
50%,99.754000,100.0,813.000000,2.000000,0.000000,1.000000,813.0,33320.000000,32694.000000,1.0,-1.000000,0.0,1491.000000
75%,100.000000,100.0,813.000000,2.000000,0.000000,1.000000,813.0,53317.000000,54129.000000,1.0,1.000000,0.0,1502.000000
max,100.000000,100.0,813.000000,3.000000,1.000000,10.000000,813.0,193811.000000,192999.000000,1.0,1.000000,0.0,1502.000000


There were 81 rows in the table, but only 77 different database sequences (i.e., plasmids) were found. This indicates that some plasmids contained multiple copies of the AMR gene.  

To confirm this, we'll need to go back and take a look at the blast results.  The next command will identify those plasmids with multiple BLAST matches and print them out.

In [8]:
blast_results[blast_results.duplicated('sseqid', False)]

,qseqid,sseqid,stitle,pident,qcovs,length,mismatch,gapopen,qstart,qend,sstart,send,qframe,sframe,frames,evalue,bitscore,qseq,sseq
20,MH168512.2,NZ_CP010373.2,Escherichia coli strain 6409 plasmid p6409-202...,100.000,100,813,0,0,1,813,184799,183987,1,-1,1/-1,0.0,1502,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...
21,MH168512.2,NZ_CP010373.2,Escherichia coli strain 6409 plasmid p6409-202...,100.000,100,813,0,0,1,813,193811,192999,1,-1,1/-1,0.0,1502,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...
22,MH168512.2,NZ_CP010373.2,Escherichia coli strain 6409 plasmid p6409-202...,100.000,100,804,0,0,10,813,8900,8097,1,-1,1/-1,0.0,1485,CCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCGCATTAGCCG...,CCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCGCATTAGCCG...
61,MH168512.2,NZ_CP025626.1,Escherichia coli strain SCEC020007 plasmid pND...,99.754,100,813,2,0,1,813,22628,21816,1,-1,1/-1,0.0,1491,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...
62,MH168512.2,NZ_CP025626.1,Escherichia coli strain SCEC020007 plasmid pND...,99.754,100,813,2,0,1,813,74846,75658,1,1,1/1,0.0,1491,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...
76,MH168512.2,NZ_CM007910.1,Escherichia coli strain CH613_eco plasmid unna...,99.754,100,813,2,0,1,813,21052,21864,1,1,1/1,0.0,1491,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...
77,MH168512.2,NZ_CM007910.1,Escherichia coli strain CH613_eco plasmid unna...,99.508,100,813,2,1,1,813,123567,124377,1,1,1/1,0.0,1478,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...,ATGGAATTGCCCAATATTATGCACCCGGTCGCGAAGCTGAGCACCG...


It looks like we have three plasmids with multiple BLAST matches.  Those three plasmids are in rows 20-22 (NZ_CP010373.2), 61-62 (NZ_CP025626.1), and 76-77 (NZ_CM007910.1). 

Looking at these results, we see that these are all strong matches.  All the matches are in excess of 99% identical (pident column) and they all cover most of the gene (length column).  The sstart and send columns identify the start and end of the alignment on the plasmid sequence.

This use of pandas was inspired by the workflow at https://github.com/fomightez/blast-binder  

In [15]:
blast_results.bitscore.min()

1478